In [1]:
#code credits: Professor Richard Hull
# These are boiler plate imports that seem useful
# I should probably delete or comment out the ones that aren't used in this script...

import sys
import os
import subprocess
import shutil
# use shutil.rmtree(<path_to_folder>) to do an rm -r on the folder

import json
import csv
import yaml

import urllib

import pandas as pd
import numpy as np

import matplotlib as mpl
from cycler import cycler

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")

# sys.path.append('/Users/rick/HACKING/UTILITIES_IN_HACKING/')
# import utils_general


# this one gives a warning - not fixing for now
# from owlready2 import *

# next function is used to generate timestamps that can be used to
# create uniquely named directories and/or files
import datetime
def curr_timestamp():
    current_datetime = datetime.datetime.now()
    formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")
    return formatted_datetime


print('Initial imports successful')

Initial imports successful


In [2]:
def pull_useful(ginfo):  # should give as input the result.stdout from calling gdalinfo -json
    useful = {}
    useful['cornerCoordinates'] = ginfo['cornerCoordinates']
    useful['proj:transform'] = ginfo['stac']['proj:transform']
    useful['size'] = ginfo['size']
    useful['bbox'] = ginfo['stac']['proj:projjson']['bbox']
    useful['espgEncoding'] = ginfo['stac']['proj:epsg']
    return useful

In [6]:
from osgeo import gdal

work_dir = "/Users/kavya/Documents/ECS289L/Project2/GAEZ/tif_files/Temp/"
pkl_file_path = '/Users/kavya/Documents/ECS289L/Project2/GAEZ/tif_files/pkl_files/temp_df.pkl'
arr_sec = list(range(1, 44))
col_names = ['sector', 'year', 'temp']

rain_df = pd.DataFrame(columns=col_names)

file_names = os.listdir(work_dir)

# Filter for only .tif files
tif_files = [file for file in file_names if file.endswith('.tif')]

for file_name in tif_files:
    tif_file_name = file_name
    year = tif_file_name.split('_')[3].replace('.tif', '')
    ds = gdal.Open(work_dir + tif_file_name)
    cmd = 'gdal_translate -projwin'
    #39.2856,-122.1515 upper left 
    #39.2896,-121.7084 upper right
    #38.6845,-121.7559 lower left
    #38.6819,-121.5623 lower right
    leftLon = '-122.1515'
    upperLat = '39.2896'
    rightLon = '-121.5623'
    lowerLat = '38.6819'
    format = '-of GTiff'
    source = work_dir + tif_file_name
    clipped_file_name = tif_file_name.replace('.tif','')+'_c.tif'
    target = work_dir + 'clipped/' + clipped_file_name

    val = " ".join([cmd, leftLon, upperLat, rightLon, lowerLat, format, source, target])
    #print(val)

    result = subprocess.run([val], shell=True, capture_output=True, text=True)

    print()
    #print(result.stdout)
    #print(result.stderr)

    val = " ".join(["gdalinfo", "-json", target])
    print(val)
    result = subprocess.run([val], shell=True, capture_output=True, text=True)

    dict = json.loads(result.stdout)

    useful = pull_useful(dict)
    #print(json.dumps(useful, indent=2, sort_keys=True))

    dsc = gdal.Open(target)
    dcarray = np.array(dsc.GetRasterBand(1).ReadAsArray())

    '''
    plt.figure(figsize=(10,5))
    plt.imshow(dcarray) 

    plt.colorbar()
    plt.show()
    '''
    
    arr_temp = []
    for row in dcarray:
        arr_temp.extend(row)
    arr_temp = arr_temp[:43]

    arr_year = [year]*43

    temp_df = pd.DataFrame({
        'sector': arr_sec,
        'year': arr_year,
        'temp': arr_prc
    })

    rain_df = pd.concat([rain_df, temp_df])
    
    # Save the DataFrame as a pickle file
    rain_df.to_pickle(pkl_file_path)


gdalinfo -json /Users/kavya/Documents/ECS289L/Project2/GAEZ/tif_files/Temp/clipped/tmp_GFDL-ESM2M_rcp8p5_2056_c.tif

gdalinfo -json /Users/kavya/Documents/ECS289L/Project2/GAEZ/tif_files/Temp/clipped/tmp_GFDL-ESM2M_rcp8p5_2042_c.tif

gdalinfo -json /Users/kavya/Documents/ECS289L/Project2/GAEZ/tif_files/Temp/clipped/tmp_GFDL-ESM2M_rcp8p5_2095_c.tif

gdalinfo -json /Users/kavya/Documents/ECS289L/Project2/GAEZ/tif_files/Temp/clipped/tmp_GFDL-ESM2M_rcp8p5_2081_c.tif

gdalinfo -json /Users/kavya/Documents/ECS289L/Project2/GAEZ/tif_files/Temp/clipped/tmp_GFDL-ESM2M_rcp8p5_2080_c.tif

gdalinfo -json /Users/kavya/Documents/ECS289L/Project2/GAEZ/tif_files/Temp/clipped/tmp_GFDL-ESM2M_rcp8p5_2094_c.tif

gdalinfo -json /Users/kavya/Documents/ECS289L/Project2/GAEZ/tif_files/Temp/clipped/tmp_GFDL-ESM2M_rcp8p5_2043_c.tif

gdalinfo -json /Users/kavya/Documents/ECS289L/Project2/GAEZ/tif_files/Temp/clipped/tmp_GFDL-ESM2M_rcp8p5_2057_c.tif

gdalinfo -json /Users/kavya/Documents/ECS289L/Project2/GAEZ/tif


gdalinfo -json /Users/kavya/Documents/ECS289L/Project2/GAEZ/tif_files/Temp/clipped/tmp_GFDL-ESM2M_rcp8p5_2060_c.tif

gdalinfo -json /Users/kavya/Documents/ECS289L/Project2/GAEZ/tif_files/Temp/clipped/tmp_GFDL-ESM2M_rcp8p5_2074_c.tif

gdalinfo -json /Users/kavya/Documents/ECS289L/Project2/GAEZ/tif_files/Temp/clipped/tmp_GFDL-ESM2M_rcp8p5_2048_c.tif

gdalinfo -json /Users/kavya/Documents/ECS289L/Project2/GAEZ/tif_files/Temp/clipped/tmp_GFDL-ESM2M_rcp8p5_2049_c.tif

gdalinfo -json /Users/kavya/Documents/ECS289L/Project2/GAEZ/tif_files/Temp/clipped/tmp_GFDL-ESM2M_rcp8p5_2075_c.tif

gdalinfo -json /Users/kavya/Documents/ECS289L/Project2/GAEZ/tif_files/Temp/clipped/tmp_GFDL-ESM2M_rcp8p5_2061_c.tif

gdalinfo -json /Users/kavya/Documents/ECS289L/Project2/GAEZ/tif_files/Temp/clipped/tmp_GFDL-ESM2M_rcp8p5_2059_c.tif

gdalinfo -json /Users/kavya/Documents/ECS289L/Project2/GAEZ/tif_files/Temp/clipped/tmp_GFDL-ESM2M_rcp8p5_2065_c.tif

gdalinfo -json /Users/kavya/Documents/ECS289L/Project2/GAEZ/tif

In [7]:
df = pd.read_pickle('/Users/kavya/Documents/ECS289L/Project2/GAEZ/tif_files/pkl_files/temp_df.pkl')
df

,sector,year,temp
0,1,2056,19.150000
1,2,2056,19.139999
2,3,2056,19.270000
3,4,2056,19.260000
4,5,2056,19.230000
...,...,...,...
38,39,2073,19.029999
39,40,2073,19.029999
40,41,2073,19.080000
41,42,2073,19.160000
